In [12]:
import pandas as pd
import polars as pl

In [3]:
#Put a switch that does this
input_path = r"C:\Users\Mert\OneDrive\Desktop\Courses\Financial Big Data\data\nasdaq_exteral_data.csv"

In [4]:
def filter_and_save_parquet(input_path: str, ticker_list):

    # Process each ticker separately to reduce memory usage
    for ticker in ticker_list:
        # Read and filter data in streaming mode
        lazy_df = pl.scan_csv(input_path)
        filtered_df = (
            lazy_df
            .select(["Date", "Stock_symbol", "Article_title"])  # Select specific columns
            .filter(pl.col("Article_title").is_not_null())  # Filter out null values
            .filter(pl.col("Stock_symbol") == ticker)  # Filter rows matching the ticker
            .with_columns(
                pl.col("Date").str.to_datetime("%Y-%m-%d %H:%M:%S UTC")  # Convert Date column to datetime
            )
        )

        # Collect filtered data and write directly to a Parquet file
        output_path = f"../raw/data/{ticker}_news.parquet"
        filtered_df.sink_parquet(output_path)  # Stream writing without converting to Pandas


In [ ]:
#Applying the function (can extract as many tickers you like)
filter_and_save_parquet(input_path = input_path, ticker_list = ["AAPL","GOOG","TSLA"])